In [1]:
import sympy as sp
from sympy.parsing.mathematica import mathematica
import pyperclip

# Default mathemathica required translations
TRANSLATIONS = {'Csc[x]': '1 / sin(x)',
                'Cot[x]': '1 / tan(x)'}


def parse_mathematica_vector(fpath, additional_translations=None, subs=None):
    r"""
    Parses a vector expression from Mathematica and translates it to sp.
    The vector must look like :math:`\left\{x_0, x_1, \ldots\right)}`, i.e.
    be wrapped in curly brackets and separated with ', '. Note the space
    following the comma.

    See documentation for more details on how to export from Mathematica.


    Arguments
    ---------
    fpath: str
        Filepath to the saved '.txt' mathematica expression.
    additional_translations: dict
        Dictionary of translations of Mathematica functions to sp. For
        example: `{'Csc[x]': '1 / sin(x)'}`. Used for functions which are not
        natively supported in sp.
    subs: dict
        Dictionary of symbolic substitutions into the final expression, can
        look like `{'alpha': alpha}`, where `alpha` is a
        `:py:class:sp.Symbol` type.

    Returns
    -------
    vector: sp.Matrix
        The translated symbolic vector.
    """
    if additional_translations is None:
        additional_translations = TRANSLATIONS
    vector = []
    with open(fpath, "r") as file:
        for i, line in enumerate(file):
            expr = line
            if i > 0:
                raise ValueError("Unexpected input (more than 1 line).")
        # Mathematica vector is split like this
        # The brackets denote the first and last position.
        for i, sym in enumerate(expr.split(', ')):
            if sym.startswith('{'):
                sym = sym[1:]
            if sym.endswith('}'):
                sym = sym[:-1]
            vector.append(mathematica(sym, additional_translations))

    vector = sp.Matrix(vector)

    if subs is not None:
        if not isinstance(subs, list):
            subs = [subs]
        for sub in subs:
            vector = vector.subs(sub)
    return vector


def sympy_to_julia(expr):
    julia_code = sp.printing.julia_code(expr)
    julia_code = julia_code.replace('.*', '*')
    julia_code = julia_code.replace('./', '/')
    julia_code = julia_code.replace('.+', '+')
    julia_code = julia_code.replace('.-', '-')
    julia_code = julia_code.replace('.^', '^')
    
    
    julia_code = julia_code.replace('GWR', 'x[2]')
    julia_code = julia_code.replace('GWTHETA', 'x[3]')
    julia_code = julia_code.replace('HXPR', 'x[5]')
    julia_code = julia_code.replace('HXPTHETA', 'x[6]')
    julia_code = julia_code.replace('HXPPHI', 'x[6]')
    julia_code = julia_code.replace('ZYA', 'p[1]')
    julia_code = julia_code.replace('ZYEPS', 'p[2]')
    julia_code = julia_code.replace('ZYS', 'p[3]')

    return julia_code

In [69]:
x_vector = sp.symbols('t r θ ϕ')
# We want to rename some variables
subs = {'p{}'.format(i): sp.Symbol('p_{}'.format(p)) for i, p in enumerate(x_vector)}
TRIG_SUBS = {sp.cos(x_vector[2]): 'c_θ',
             sp.sin(x_vector[2]): 's_θ',
             sp.cos(2*x_vector[2]): 'c_2θ',
             sp.sin(2*x_vector[2]): 's_2θ',
             sp.tan(x_vector[2]): 't_θ',
             sp.tan(2*x_vector[2]): 't_2θ'}


vector_subs = {'r': 'GWR',
               'θ': 'GWTHETA',
               'p_r': 'HXPR',
               'p_θ': 'HXPTHETA',
               'p_ϕ': 'HXPPHI',
               'a': 'ZYA',
               '}': 'ZYEPS',
               's': 'ZYS'}

subs.update(TRIG_SUBS)

subs.update({'Z1': 'Robs',
             'Z2': 'θobs',
             'v3': 'v3',
             'dv3': 'dv3',
             'Y1': 'Rsrc',
             'Y2': 'θsrc'})

odes = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_EQS.txt",
                                subs=[subs])
geodesic_odes = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_EQSgeodesic.txt",
                                subs=[subs])
pi0 = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_pinitial.txt",
                               subs=[subs])[1:]
pi0 = sp.Matrix(pi0)
p0 = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_pt0EQ.txt",
                              subs=[subs])[0]

v3 = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_v0.txt",
                              subs=[subs])[0]

dv3dr = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_v1.txt",
                              subs=[subs])[0]

static_observer_proper_time = parse_mathematica_vector("/Users/richard/Projects/birefringence/mathematica_export/Kerr_stat_obs_propertime.txt",
                                                       subs=[subs])[0]

In [84]:
expr = sympy_to_julia(odes[6])
print(expr)
pyperclip.copy(expr)

2*a^3*c_θ*r*s_θ^3*(p_θ/(a^2*c_θ^2 + r^2) + ϵ*(p_r*s*s_θ^2*(2*a*p_θ*(-2*a*c_θ*s_θ*(a^2*c_θ^2 + r^2)*(a*s_θ*v3*sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r)) + sqrt(a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r))/sqrt(-(v3^2 - 1)*(a^2 + r^2 - 2*r)) - a*r*s_2θ*(a*s_θ*v3*sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r)) + sqrt(a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r))/sqrt(-(v3^2 - 1)*(a^2 + r^2 - 2*r)) + 2*c_θ*r*s_θ*(-2*a*r*(a*s_θ*v3/sqrt(a^2*c_θ^2 + r^2) + (a^2 + r^2)/sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r))) + (a/sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r)) + v3/(s_θ*sqrt(a^2*c_θ^2 + r^2)))*(-a^2*s_θ^2*(a^2 + r^2 - 2*r) + (a^2 + r^2)^2))/sqrt(1 - v3^2) - c_θ*(a^2*c_θ^2 + r^2)^2*(-a^2*s_θ^2*v3*(a^2 + r^2 - 2*r)/sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r)) - a*s_θ*(a^2 + r^2 - 2*r)/sqrt(a^2*c_θ^2 + r^2) + v3*sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r)))/sqrt(-(v3^2 - 1)*(a^2 + r^2 - 2*r))) + p_r*(a^2 + r^2 - 2*r)*(4*r*(a^2*c_θ^2 + r^2)*(a*s_θ*v3*sqrt((a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r)) + sqrt(a^2*c_θ^2 + r^2)*(a^2 + r^2 - 2*r